<a href="https://colab.research.google.com/github/jose98orellana/CPSC501A4/blob/master/CHDModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
https://www.tensorflow.org/tutorials/load_data/csv
https://www.tensorflow.org/tutorials/keras/overfit_and_underfit
https://www.tensorflow.org/api_docs/python/

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import functools

import sys
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Upload files

In [0]:
from google.colab import files
uploaded = files.upload()

Saving heart_test.csv to heart_test.csv
Saving heart_train.csv to heart_train.csv


Process files

In [0]:
import io
train_file = pd.read_csv(io.BytesIO(uploaded['heart_train.csv']))
test_file = pd.read_csv(io.BytesIO(uploaded['heart_test.csv']))
# Dataset is now stored in a Pandas Dataframe

In [0]:
train_file['famhist'] = pd.Categorical(train_file['famhist'])
train_file['famhist'] = train_file.famhist.cat.codes
test_file['famhist'] = pd.Categorical(test_file['famhist'])
test_file['famhist'] = test_file.famhist.cat.codes

In [0]:
train_file.head()

In [0]:
target = train_file.pop('chd')

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((train_file.values, target.values))

In [0]:
for feat, targ in dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

In [0]:
tf.constant(train_file['famhist'])

In [0]:
test_dataset = dataset.shuffle(len(test_file)).batch(1)
train_dataset = dataset.shuffle(len(train_file)).batch(1)

Create and train a model

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

model = get_compiled_model()
model.fit(train_dataset, epochs=15)

In [0]:
print("--Evaluate model--")
model_loss, model_acc = model.evaluate(test_dataset)
print(f"Model Test Loss:    {model_loss:.2f}")
print(f"Model Test Accuray: {model_acc*100:.1f}%")

--Evaluate model--
385/385 [==============================] - 0s 971us/step - loss: 0.6346 - accuracy: 0.6675
Model Test Loss:    0.63
Model Test Accuray: 66.8%
